In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("tips.csv")

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   size              244 non-null    int64  
 7   price_per_person  244 non-null    float64
 8   Payer Name        244 non-null    object 
 9   CC Number         244 non-null    int64  
 10  Payment ID        244 non-null    object 
dtypes: float64(3), int64(2), object(6)
memory usage: 21.1+ KB


In [6]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person', 'Payer Name', 'CC Number', 'Payment ID'],
      dtype='object')

In [7]:
df.describe()

,total_bill,tip,size,price_per_person,CC Number
count,244.000000,244.000000,244.000000,244.000000,2.440000e+02
mean,19.785943,2.998279,2.569672,7.888197,2.563496e+15
std,8.902412,1.383638,0.951100,2.914234,2.369340e+15
min,3.070000,1.000000,1.000000,2.880000,6.040679e+10
25%,13.347500,2.000000,2.000000,5.800000,3.040731e+13
50%,17.795000,2.900000,2.000000,7.255000,3.525318e+15
75%,24.127500,3.562500,3.000000,9.390000,4.553675e+15
max,50.810000,10.000000,6.000000,20.270000,6.596454e+15


##  En yüksek 5 toplam hesaba (total_bill) ve ilişkili bahşişlere bak.

In [11]:
df.nlargest(5, 'total_bill').loc[:, ['total_bill' , 'tip' , 'sex' , 'day']]

,total_bill,tip,sex,day
170,50.81,10.00,Male,Sat
212,48.33,9.00,Male,Sat
59,48.27,6.73,Male,Sat
156,48.17,5.00,Male,Sun
182,45.35,3.50,Male,Sun


## Günlere göre (day) toplam hesaptaki (total_bill) ortalamayı artan sırada listele.


In [17]:
df.groupby('day' , as_index=False)['total_bill'].mean().sort_values('total_bill')\
                                                .reset_index(drop=True).rename(columns={
                                                            'total_bill':'ortalama_hesap'   
                                                                                })

,day,ortalama_hesap
0,Fri,17.151579
1,Thur,17.682742
2,Sat,20.441379
3,Sun,21.410000


## Hafta sonu (Sat, Sun) ile hafta içi (Thur, Fri) ortalama bahşişleri ve toplam kişi sayısını pivot tablo ile göster.

In [18]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251


In [ ]:
df['weektype'] = df['day'].apply(lambda x: 'HaftaSonu' if x in ['Sat' , 'sun'] else 'Haftaİçi')

pd.pivot_table(data=df , index='weektype' ,  values=['tip' , 'size'],
                                             aggfunc={'tip':'mean',
                                                     'size':'sum'} )

,size,tip
weektype,,
HaftaSonu,219,2.993103
Haftaİçi,408,3.001146


## Kadın ve erkek müşterilerin ortalama bahşiş oranlarını (tip/total_bill) karşılaştır, en çok bahşişi kim veriyor?

In [27]:
(df['tip'] / df['total_bill']).groupby(df['sex']).mean()

sex
Female    0.166491
Male      0.157651
dtype: float64

## Bahşiş oranı %20’den fazla olan kayıtların gün dağılımı

In [28]:
df[df['tip'] / df['total_bill'] > 0.20]['day'].value_counts()

day
Sun     15
Sat     10
Thur    10
Fri      4
Name: count, dtype: int64

## 3+ kişiyle gelen masalarda, en yüksek ort. toplam hesabı olan günü bul

In [29]:
df[df['size']>=3].groupby('day')['total_bill'].mean().idxmax()

'Fri'

## Hafta sonu içenlerle hafta içi içmeyenlerin ort. bahşiş oranı

In [31]:
df['tip_rate'] = df['tip'] / df['total_bill']
df[(df['day'].isin(['Sat','Sun'])) & (df['smoker'] == 'Yes')]['tip_rate'].mean(), \
df[(df['day'].isin(['Thur','Fri'])) & (df['smoker'] == 'No')]['tip_rate'].mean()

(np.float64(0.16016083188625413), np.float64(0.15959215195306276))

## En yüksek ve en düşük bahşiş oranlı 3 müşteri

In [34]:
df.assign(tip_rate=df['tip']/df['total_bill']).sort_values('tip_rate')

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,weektype,tip_ratio,ort,tip_rate
237,32.83,1.17,Male,Yes,Sat,Dinner,2,16.42,Thomas Brown,4284722681265508,Sat2929,HaftaSonu,0.035638,0.035638,0.035638
102,44.30,2.50,Female,Yes,Sat,Dinner,3,14.77,Heather Cohen,379771118886604,Sat6240,HaftaSonu,0.056433,0.056433,0.056433
57,26.41,1.50,Female,No,Sat,Dinner,2,13.20,Melody Simon,4745394421258160,Sat8980,HaftaSonu,0.056797,0.056797,0.056797
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,Haftaİçi,0.059447,0.059447,0.059447
187,30.46,2.00,Male,Yes,Sun,Dinner,5,6.09,David Barrett,4792882899700988,Sun9987,Haftaİçi,0.065660,0.065660,0.065660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,23.17,6.50,Male,Yes,Sun,Dinner,4,5.79,Dr. Michael James,4718501859162,Sun6059,Haftaİçi,0.280535,0.280535,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,5.80,James Taylor,6011482917327995,Sat2124,HaftaSonu,0.291990,0.291990,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455,HaftaSonu,0.325733,0.325733,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,4.80,Melanie Gray,4211808859168,Sun4598,Haftaİçi,0.416667,0.416667,0.416667


## Her gün ve time için toplam hesap, toplam kişi, ort. oranı

In [36]:
df.groupby(['day' , 'time']).agg({'total_bill':'sum',
                                  'size':'sum',
                                  })

total_bill  size
day  time                    
Fri  Dinner      235.96    26
     Lunch        89.92    14
Sat  Dinner     1778.40   219
Sun  Dinner     1627.16   216
Thur Dinner       18.78     2
     Lunch      1077.55   150